In [ ]:
from helper import get_azure_openai_keys

AZURE_API_KEY, AZURE_ENDPOINT, AZURE_API_VERSION = get_azure_openai_keys()

In [ ]:
import nest_asyncio

nest_asyncio.apply()

# simple tools

In [ ]:
from llama_index.core.tools import FunctionTool


def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y


def mystery(x: int, y: int) -> int:
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [ ]:
from helper import get_azure_llm

llm = get_azure_llm()
response = llm.predict_and_call(
    [add_tool, mystery_tool],
    "Tell me the output of the mystery function on 2 and 9",
    verbose=True,
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


# auto retrieval tool

## load data

In [ ]:
from helper import get_azure_llm, get_azure_embed_model

llm = get_azure_llm()
embed_model = get_azure_embed_model()

In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["../data/metagpt.pdf"]).load_data()

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [ ]:
print(nodes[30].get_content(metadata_mode="all"))

page_label: 26
file_name: metagpt.pdf
file_path: ../data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-01-02
last_modified_date: 2025-01-02

Preprint
Figure 11: The system interface design for “recommendation engine development” is generated by
the architect agent (zoom in for a better view).
E M ORE DISCUSSIONS
E.1 D EEP -SEATED CHALLENGES
MetaGPT also alleviates or solves these challenges with its unique designs:
Use Context Efficiently Two sub-challenges are present. First, unfolding short natural language
descriptions accurately to eliminate ambiguity. Second, maintaining information validity in lengthy
contexts, enables LLMs to concentrate on relevant data without distraction.
Reduce Hallucinations Using LLMs to generate entire software programs faces code halluci-
nation problems—-including incomplete implementation of functions, missing dependencies, and
potential undiscovered bugs, which may be more serious. LLMs often struggle with software gen

In [ ]:
for i, node in enumerate(nodes):
    print(f"{i} {node.get_metadata_str(mode='embed')[:14]}")

0 page_label: 1

1 page_label: 2

2 page_label: 3

3 page_label: 3

4 page_label: 4

5 page_label: 5

6 page_label: 6

7 page_label: 7

8 page_label: 7

9 page_label: 8

10 page_label: 9

11 page_label: 10
12 page_label: 10
13 page_label: 11
14 page_label: 11
15 page_label: 12
16 page_label: 12
17 page_label: 13
18 page_label: 14
19 page_label: 15
20 page_label: 16
21 page_label: 17
22 page_label: 18
23 page_label: 19
24 page_label: 20
25 page_label: 21
26 page_label: 22
27 page_label: 23
28 page_label: 24
29 page_label: 25
30 page_label: 26
31 page_label: 27
32 page_label: 28
33 page_label: 29


In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [ ]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts([{"key": "page_label", "value": "2"}]),
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?",
)

In [ ]:
print(str(response))

MetaGPT achieves a new state-of-the-art with 85.9% and 87.7% in Pass@1 for code generation benchmarks. It also demonstrates a 100% task completion rate, showcasing its robustness and efficiency in handling complex software projects.


In [ ]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '../data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}


## define auto retrieval tool

In [ ]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(query: str, page_numbers: List[str]) -> str:
    """Perform a vector search over an index.

    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.

    """

    metadata_dicts = [{"key": "page_label", "value": p} for p in page_numbers]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts, condition=FilterCondition.OR
        ),
    )
    response = query_engine.query(query)
    return response


vector_query_tool = FunctionTool.from_defaults(name="vector_tool", fn=vector_query)

In [ ]:
llm = get_azure_llm()

response = llm.predict_and_call(
    [vector_query_tool],
    "What are the high-level results of MetaGPT as described on page 2?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves a new state-of-the-art with 85.9% and 87.7% in Pass@1 on code generation benchmarks. It also demonstrates a 100% task completion rate, showcasing its robustness and efficiency in handling complex software projects.


In [ ]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '../data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}


# and with other tools

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=("Useful if you want to get a summary of MetaGPT"),
)

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What are the MetaGPT comparisons with ChatDev described on page 8?",
    verbose=True,
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev in several aspects on the SoftwareDev dataset. MetaGPT achieves a higher executability score of 3.75 compared to ChatDev's 2.25. It also takes less time to run (503 seconds versus 762 seconds) and requires fewer tokens to generate one line of code (126.5/124.3 tokens compared to ChatDev's 248.9 tokens). Additionally, MetaGPT produces more code files and lines of code per file, resulting in a higher total number of code lines. The cost of human revision is also significantly lower for MetaGPT (0.83) compared to ChatDev (2.5).


In [ ]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '8', 'file_name': 'metagpt.pdf', 'file_path': '../data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2025-01-02', 'last_modified_date': '2025-01-02'}


In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], "What is a summary of the paper?", verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "MetaGPT"}
=== Function Output ===
MetaGPT is a meta-programming framework designed to enhance multi-agent collaboration in software development using large language models (LLMs). It employs a structured division of labor among specialized agents, such as Product Managers, Architects, Engineers, and QA Engineers, to streamline workflows and reduce errors in complex tasks. MetaGPT follows Standardized Operating Procedures (SOPs) and uses documents and diagrams for communication, rather than dialogue, to ensure clarity and consistency. The framework incorporates mechanisms for self-improvement and iterative feedback to improve code quality and productivity. It has demonstrated superior performance in benchmarks like HumanEval and MBPP, showcasing its robustness and efficiency in handling complex software engineering projects. MetaGPT also supports user data privacy by operating locally and not collecting user d